In [2]:
import os
import shutil
from monai.utils import first, set_determinism, progress_bar
import SimpleITK as sitk  # noqa: N813
import numpy as np
import itk
import tempfile
import monai
from monai.data import PILReader
from monai.transforms import (LoadImage, LoadImaged,ScaleIntensityRangePercentiles, Resized, Compose, SaveImage, 
ScaleIntensity,RandScaleIntensity, SpatialPadd)
from monai.config import print_config
from monai.data import CacheDataset, NiftiSaver
import glob
import torch
import nibabel as nib
from monai.transforms import (
    Activations,
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    NormalizeIntensityd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    Invertd,
    RandAffined,
    SpatialPadd,
    Rand3DElasticd,
    RandFlipd,
    ResizeWithPadOrCropd
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss, DiceCELoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch, PatchDataset
from monai.config import print_config
from monai.apps import download_and_extract
from monai.inferers import sliding_window_inference
import matplotlib.pyplot as plt

In [3]:
cd

/home/athurai3


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    dropout = 0.4,
    norm=Norm.BATCH,
).to(device)

In [5]:
test_ct = ['projects/ctb-akhanf/athurai3/unet_contact_seg/sample_data/sub-P003/sub-P003_desc-rigid_space-T1w_znorm_ct.nii.gz']

test_data = [{"image": image} for image in test_ct]

test_transforms = Compose( #loading full image
    [
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"])])

test_ds = Dataset(data = test_data, transform = test_transforms)

test_loader = DataLoader(test_ds, batch_size = 1, num_workers = 1)


post_transforms = post_transforms = Compose(
    [
        Activations(sigmoid = True),
        AsDiscrete(threshold = 0.5),
        #SaveImaged(keys="pred", meta_keys="pred_meta_dict", output_dir=monai_test_dir, resample=False),
    ]
)
post_pred = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
date = '2024Feb5'
pred_imgs = []
model.load_state_dict(torch.load(f'/home/athurai3/scratch/seeg_contacts_loc/derivatives/UNET/{date}_diceloss_checkpoint.pt', map_location=torch.device('cpu')))
model.eval()
with torch.no_grad():
    for test_img in test_loader:
        test_inputs = test_img['image'].to('cpu')
        roi_size = (32, 32, 32)
        sw_batch_size = 5
        pred_img = sliding_window_inference(inputs = test_inputs, roi_size=roi_size, sw_batch_size = sw_batch_size, predictor = model.to('cpu'),
                                            overlap = 0.15, mode = 'constant', sw_device = 'cpu', device = 'cpu', progress=True)
        pred_imgs.append(pred_img)

100%|██████████| 650/650 [05:57<00:00,  1.82it/s]


In [6]:
pred_imgs = post_transforms(pred_imgs)

In [7]:
torch.unique(pred_imgs[0][0][0])

tensor([0., 1.])

In [8]:
prediction = pred_imgs[0][0][0]
file_name_pred = 'scratch/sub-P003_desc-rigid_space-T1w_ct_pred.nii.gz'
orig_test = nib.load('projects/ctb-akhanf/athurai3/unet_contact_seg/sample_data/sub-P003/sub-P003_desc-rigid_space-T1w_znorm_ct.nii.gz')
file = nib.Nifti1Image(prediction.detach().numpy(), affine = orig_test.affine, header = orig_test.header)
nib.save(file, file_name_pred)

In [ ]:
fname = 'projects/ctb-akhanf/athurai3/unet_contact_seg/sample_data/sub-P001/sub-P001_desc-clipped_patches.dat'
bps = 4 * num_channels * np.prod(dims)         # Bytes per sample
file_size = os.path.getsize(fname) 
num_samples = np.floor_divide(file_size,bps)   # Number of samples
print(file_size)
print(bps)
print(num_samples)


#can change first index from num_samples to try with training on smaller number of jobs, kjupyter job
dims = dims.astype('int')
arr_shape_train = (num_samples, dims[0],dims[1],dims[2],num_channels)

arr_train = np.memmap(fname,'float32','r',shape=arr_shape_train)
#arr_train = np.swapaxes(arr_train,1,3)
print(arr_shape_train)

arr_train = np.swapaxes(arr_train,1,4)
arr_train_image = arr_train[:,0,:,:,:].reshape(arr_train.shape[0],1,arr_train.shape[2],arr_train.shape[3],arr_train.shape[4])
arr_train_label = arr_train[:,1,:,:,:].reshape(arr_train.shape[0],1,arr_train.shape[2],arr_train.shape[3],arr_train.shape[4])

thresh_train_label = np.where(arr_train_label[:][:] > 0, 1, arr_train_label)

arr_train_dict= [{"image": ct, "label": contact} for ct, contact in zip(arr_train_image,thresh_train_label)]

